# Assignment 3

### <span style="color:chocolate"> Submission requirements </span>

Your work will not be graded if your notebook doesn't include output. In other words, <span style="color:red"> make sure to rerun your notebook before submitting to Gradescope </span> (Note: if you are using Google Colab: go to Edit > Notebook Settings  and uncheck Omit code cell output when saving this notebook, otherwise the output is not printed).

Additional points may be deducted if these requirements are not met:
    
* Comment your code;
* Each graph should have a title, labels for each axis, and (if needed) a legend. Each graph should be understandable on its own;
* Try and minimize the use of the global namespace (meaning, keep things inside functions).
---

### Import libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style
from sklearn.model_selection import train_test_split
import tensorflow as tf
from matplotlib import pyplot as plt

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, RegressorMixin

This lab continues our study of linear regression. You'll train your first models with Tensorflow, using a real dataset to predict car prices from their features. Note that Tensorflow is a rapidly changing library. This means you'll often see warnings about deprecations. You can ignore the warnings in our labs.

---
### Step 1: Data ingestion

You'll use the [Automobile Data Set](https://archive.ics.uci.edu/ml/datasets/automobile)  from 1985 Ward's Automotive Yearbook that is part of the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets).

In [ ]:
# Provide the names for the feature columns since the CSV file with the data
# does not have a header row.
cols = ['symboling', 'losses', 'make', 'fuel-type', 'aspiration', 'num-doors',
        'body-style', 'drive-wheels', 'engine-location', 'wheel-base',
        'length', 'width', 'height', 'weight', 'engine-type', 'num-cylinders',
        'engine-size', 'fuel-system', 'bore', 'stroke', 'compression-ratio',
        'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']

# Load the data from a CSV file into a pandas dataframe. Remember that each row
# is an example and each column in a feature.
car_data_init = pd.read_csv(
    'https://storage.googleapis.com/ml_universities/cars_dataset/cars_data.csv',
    sep=',', names=cols, header=None, encoding='latin-1')

# Display top five rows
print('Shape of data:', car_data_init.shape)
car_data_init.head()

---
### Step 2: Data preprocessing

This step is essential for preparing the data in a format that is suitable for ML algorithms. It helps ensure data quality and improvements in model performance.

### <span style="color:chocolate">Exercise 1:</span> Column selection (5 points)

To keep things simple, you will:

1. Retain only the following columns: ['horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']. Name the new dataframe *car_data*.
2. Display the data type of each column;
3. Convert the data type of each columns to numeric. Coerce missing values to NaN. Hint: use <span style="color:chocolate">pd.to_numeric()</span> method;
4. Display the data type of each column after the transformation performed at point 3.

In [ ]:
# made a list of column names
cols_to_keep = ['horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']

# made a new df with only the columns that we want by using our list as a filter
car_data = car_data_init[cols_to_keep]

# output type of data each column holds
print(car_data.dtypes)

# used the apply method to "apply" to_numeric to every column of our df and turned any error in 'Nan"
car_data = car_data.apply(pd.to_numeric, errors='coerce')

# output type of data each column holds
print(car_data.dtypes)

### <span style="color:chocolate">Exercise 2:</span> Example (row) selection (5 points)

To keep things simple again, you will:

1. Print the shape of the car_data;

2. Remove examples (rows) that have missing value(s). Note that in doing so, you will overwrite the car_data dataset. You should end up with 199 examples after this cleaning.

3. Print the shape of the car_data again.

It's important to acknowledge that there are multiple approaches to handling missing features, and simply discarding examples with any missing feature, though straightforward, may not be the most optimal solution. However, for the sake of simplicity, you will implement this strategy in this assignment.

In [ ]:
# print car_data shape
print(car_data.shape)

# remove example rows with Nan values
cleaned_car_data = car_data.dropna()

# print car_data shape
print(cleaned_car_data.shape)

### <span style="color:chocolate">Exercise 3:</span> Data shuffling (10 points)

Since you'll be using Batch Gradient Descent (BGD) for training, it is important that **each batch is a random sample of the data** so that the gradient computed is representative. Note that the original data (above) appears sorted by *make* in alphabetic order.

Using NumPy and Pandas methods:

1. Create a list of indices corresponding to the rows in the car_data dataset. Call this list *indices*. Print this list;

2. Shuffle *indices* using the <span style="color:chocolate">np.random.permutation()</span> method. Call the resulting array *shuffled_indices*. Print this array;
    
3. Use the method <span style="color:chocolate">dataframe.reindex()</span> to change the ordering of the car_data dataset based on the order in the *shuffled_indices* array. Note that in doing so, you will overwrite the original dataset. Print the top 5 rows.

In [ ]:
np.random.seed(0)
# Create a list of indices corresponding to the rows in df
indices = cleaned_car_data.index.tolist()

#note that rows keep their original indice number even after rows are dropped
print(indices)

# we will shuffel the list of lindices "randomly"
shuffled_indices = np.random.permutation(indices)

print(shuffled_indices)

# we made a new df and set that equal to cleaned_car_data but ordered by the random indeces we just shuffeled
indieced_clened_car_data = cleaned_car_data.reindex(shuffled_indices)

print(indieced_clened_car_data.head())

### <span style="color:chocolate">Exercise 4:</span> Define outcome and features (5 points)

Create two dataframes as follows:

1. The first dataframe contains our outcome of interest: ['price']. Note, this is what we are aiming to predict. Name this dataframe Y. Print shape of Y.
2. The second dataframe contains our features of interest: ['horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg']. Name this dataframe X. Print shape of X.


In [ ]:
#  Created a new df containing prices
Y = indieced_clened_car_data['price']
print(Y.shape)

#  Created a new df containing 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg'
X = indieced_clened_car_data[['horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg']]
print(X.shape)

### <span style="color:chocolate">Exercise 5:</span> Data splits (10 points)

Using the <span style="color:chocolate">train_test_split()</span> method available in scikit-learn:
1. Partition the (X, Y) data into training, validation, and test sets using a splitting rule of [60%, 20%, 20%], with a random state set to 1234. Name the resulting dataframes as follows: X_train, X_val, X_test, Y_train, Y_val, Y_test. Hint: To create these three partitions you will utilize the train_test_split() method twice. You should obtain [119, 40, 40] examples for training, validation, and test, respectively.
2. Print the shape of each dataframe.

Note: The validation set is crucial for evaluating different hyperparameter configurations and selecting those that yield optimal model performance. This approach avoids utilizing the test dataset during model training, as it is assumed to be "unknown" at that stage.

In [ ]:
# YOUR CODE HERE
X_train_val, X_test, Y_train_val, Y_test =  train_test_split(X, Y, test_size = 0.2, random_state = 1234)

#25% of 80% = 20% of total
X_train, X_val, Y_train, Y_val =  train_test_split(X_train_val, Y_train_val, test_size = 0.25, random_state = 1234)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_val.shape)
print(Y_test.shape)

### <span style="color:chocolate">Exercise 6:</span> Data standardization (10 points)

With this concept in mind, complete the following tasks:

1. Output the quantile values (0.25, 0.5, 0.75, 0.95) for all features in the X_train dataset. Are these values uniformly scaled across features?

2. Standardize all features in X_train, X_val, and X_test. Label the resulting dataframes as X_train_std, X_val_std, and X_test_std, respectively. Hint: standardize the validation and test data using the mean and standard deviation computed from the training data. Why?

3. Similar to point 2. but now standardize the outcome variable. Label the resulting dataframes as Y_train_std, Y_val_std, and Y_test_std.

In [ ]:
# Used the quantile method to output quantile values of all features
quantiles = X_train.quantile([0.25, 0.5, 0.75, 0.95])
print(quantiles)

# Standardize all features so validation data is scaled in the same way as our training data.
mean = X_train.mean()
std = X_train.std()

X_train_std = ( X_train - mean) / std
X_val_std = ( X_val - mean) / std
X_test_std = ( X_test - mean) / std

Y_mean = Y_train.mean()
Y_std = Y_train.std()

Y_train_std = (Y_train - Y_mean) / Y_std
Y_val_std = (Y_val - Y_mean) / Y_std
Y_test_std = (Y_test - Y_mean) / Y_std

# We standardize all the data because we need to have them usung the same scale to help maintain the relationship between the data points, otherwise it would be like comparing apples to oranges

---
### Step 3: Exploratory data analysis (EDA)

EDA plays a very important role in ML. The goal here is to develop a good understanding of our dataset, identify any data quality issues, understand patterns and relationships, which in turn, aids in subsequent modeling and interpretations.

### <span style="color:chocolate">Exercise 7:</span> Scatterplot matrix (10 points)

In this exercise you will use some simple yet useful techniques to visualize the distribution of the data.

Let's start with:

1. A scatterplot matrix to visualize the pair-wise correlations between different features and outcome in the (X_train_std, Y_train_std) data. You will use the <span style="color:chocolate">sns.pairplot()</span> method from the seaborn library imported at the top of the notebook;
2. Is any of the variables in the data normally distributed? Is it necessary for the explanatory or target variable to be normally distributed in order to train a ML model?

In [ ]:
# Make a data frame with the X_train_std, Y_train_std data we just created
train_std = X_train_std.copy()
train_std['Y'] = Y_train_std

#plot
sns.pairplot(train_std)

# In our data, none of the variables seem to be normally distributed. Fortunately for us, this is not a prerequisite for the explanatory or target variable to be normally distributed in order to train our ML model

### <span style="color:chocolate">Exercise 8:</span> Correlation matrix (10 points)

In this exercise you will:

1. Plot a correlation matrix in the form of a heatmap to visualize the linear relationships between different features and outcome in the (X_train_std, Y_train) data. Hint: this example here is very useful: https://seaborn.pydata.org/examples/many_pairwise_correlations.html
    
2. Answer the following questions:
 - Which two features are likely to be most redundant?
 - Which feature is likely to be least useful for predicting price?

In [ ]:
# Plotting correlation matrix with data from the X_train_std, Y_train data
correlation_df = X_train_std.copy()
correlation_df['Y'] = Y_train

# Compute the correlation matrix
corr = correlation_df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

sns.heatmap(corr, annot = True, fmt = '.1f', mask = mask, linewidths = 0.05, vmax=.3, center=0, cmap = 'coolwarm')

# Which two features are likely to be most redundant? - high correlation with each other, indicating that they provide similar information. -
#city-mpg and highway-mpg are most redundant because of theri score of -.8
# Which feature is likely to be least useful for predicting price? - look for the feature with the lowest absolute correlation with Y. -
# the frature least likely to be useful for predicting price is peak-rpm: This feature has a correlation of -0.1

---
### Step 4: Modeling

### <span style="color:chocolate">Exercise 9:</span> Baseline model (5 points)

Let's start by evaluating a baseline model. Precisely, you'll use the average price of cars in the training set as our baseline model -- that is, the baseline always predicts the average price regardless of the input.

1. Implement this baseline using the Y_train_std data and print the average price. Note: You can revert the price variable to the original scale for interpretation purposes.

In [ ]:
# YOUR CODE HERE
average_price_std = Y_train_std.mean()

Y_ave_price = (average_price_std * Y_std)  + Y_mean

#create a list where each element is the average price
baseline_predictions = [Y_ave_price] * len(Y_test)

print(baseline_predictions)

### <span style="color:chocolate">Exercise 10:</span> Improvement over Baseline with TensorFlow (10 points)

Let's train a linear regression model much like we did in the previous assignment, but this time using TensorFlow.

1. Fill in the <span style="color:green">NotImplemented</span> parts of the build_model() function below by following the instructions provided as comments. Hint: refer to Demo 3 in [bCourses/Modules/Live Session Demos](https://bcourses.berkeley.edu/courses/1534588/files/88733489?module_item_id=17073646) for an example.
2. Build and compile a model using the build_model() function and the (X_train_std, Y_train_std) data. Set learning_rate = 0.0001. Call the resulting object *model_tf*.
3. Train *model_tf* using the (X_train_std, Y_train_std) data. Set num_epochs = 5. Pass the (X_val_std, Y_val_std) data for validation. Hint: see the documentation behind the [tf.keras.Model.fit()](https://bcourses.berkeley.edu/courses/1534588/files/88733489?module_item_id=17073646) method.
3. Generate a plot with the loss values on the y-axis and the epoch number on the x-axis for visualization. Make sure to include axes name and title. Hint: check what the [tf.keras.Model.fit()](https://bcourses.berkeley.edu/courses/1534588/files/88733489?module_item_id=17073646) method returns.

More notes on point 1: the idea is to build a *computational graph* for linear regression, and then send data through it. There are many ways to build graphs, but [TenforFlow Keras API](https://www.tensorflow.org/api_docs/python/tf/keras) is recommended.

In [ ]:
def build_model(num_features, learning_rate):
  """Build a TF linear regression model using Keras.

  Args:
    num_features: The number of input features.
    learning_rate: The desired learning rate for SGD.

  Returns:
    model: A tf.keras model (graph).
  """
  # This is not strictly necessary, but each time you build a model, TF adds
  # new nodes (rather than overwriting), so the colab session can end up
  # storing lots of copies of the graph when you only care about the most
  # recent. Also, as there is some randomness built into training with SGD,
  # setting a random seed ensures that results are the same on each identical
  # training run.
  tf.keras.backend.clear_session()
  tf.random.set_seed(0)

  # Build a model using keras.Sequential. While this is intended for neural
  # networks (which may have multiple layers), we want just a single layer for
  # linear regression.
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(
      units = 1,        # output dim
      input_shape = (num_features,), # input dim
      use_bias = True,               # use a bias (intercept) param
      kernel_initializer = tf.keras.initializers.ones,  # initialize params to 1
      bias_initializer = tf.keras.initializers.ones,    # initialize bias to 1
  ))

  # We need to choose an optimizer. We'll use GD, which is actually mini-batch GD
  optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate)

  # Finally, compile the model. This finalizes the graph for training.
  # We specify the loss and the optimizer above
  model.compile(optimizer = optimizer, loss = 'mse')

  return model

In [ ]:
tf.random.set_seed(0)
# 2. Build and compile model  with (X_train_std, Y_train_std) data.

num_o_features = X_train_std.shape[1]
model_tf = build_model(num_features=num_o_features, learning_rate=0.0001)
history = model_tf.fit(X_train_std, Y_train_std, batch_size=32, epochs=5, validation_data=(X_val_std, Y_val_std))

#4. Generate a plot
#extract loss values & val loss
training_loss = history.history['loss']
val_loss = history.history.get('val_loss')

#create a figure
plt.figure()

#plot loss
plt.plot(training_loss, label='Training Loss')

if val_loss:
    plt.plot(val_loss, label='Validation Loss')

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.legend()
plt.show()


---
### Step 5: Hyperparameter tuning

Hyperparameter tuning is a crucial step in optimizing ML models. It involves systematically adjusting hyperparameters such as learning rate, number of epochs, and optimizer to find the model configuration that leads to the best generalization performance.

This tuning process is typically conducted by monitoring the model's performance on the validation vs. training set. It's important to note that using the test set for hyperparameter tuning can compromise the integrity of the evaluation process by violating the assumption of "blindness" of the test data.

### <span style="color:chocolate">Exercise 11:</span> Hyperparameter tuning (10 points)

1. Fine-tune the hyperparameters of *model_tf* to determine the setup that yields the most optimal generalization performance. Feel free to explore various values for the hyperparameters. Hint: ask your instructors and TAs for help if in doubt.

After identifying your preferred model configuration, print the following information:

2. The learned parameters of the model (this should include the bias term). Hint: use  <span style="color:chocolate">model_tf.layers[0].get_weights()</span>.
3. The loss at the final epoch on both the training and validation datasets;
4. The percentage difference between the losses observed on the training and validation datasets.


Please note that we will consider 'optimal model configuration' any last-epoch loss that is below 0.35.

In [ ]:
tf.random.set_seed(0)
from tqdm import tqdm

# YOUR CODE HERE
# I didnt want to manually find the best hyperparameters so I looked online how to automate it
# Define the refined hyperparameter grid
param_grid = {
    'learning_rate': [0.01, 0.001, 0.0001, 0.02, 0.002, 0.0002, 0.05, 0.005, 0.0005],
    'epochs': [5, 10, 25, 50, 100],
    'batch_size': [16, 32, 64]
}

# Function to build the model
def build_model(num_features, learning_rate=0.01):
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(
        units=1,
        input_shape=(num_features,),
        use_bias=True,
        kernel_initializer=tf.keras.initializers.Ones(),
        bias_initializer=tf.keras.initializers.Ones(),
    ))
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_std)
X_val_scaled = scaler.transform(X_val_std)

num_features = X_train_std.shape[1]

best_params = None
best_val_loss = float('inf')
best_model = None

# Calculate total number of iterations for progress bar
total_iterations = len(param_grid['learning_rate']) * len(param_grid['epochs']) * len(param_grid['batch_size'])

# Iterate through the hyperparameter grid with a progress bar
with tqdm(total=total_iterations) as pbar:
    for lr in param_grid['learning_rate']:
        for epochs in param_grid['epochs']:
            for batch_size in param_grid['batch_size']:
                # Build and compile the model
                model = build_model(num_features=num_features, learning_rate=lr)

                # Train the model
                history = model.fit(X_train_scaled, Y_train_std, epochs=epochs, batch_size=batch_size, validation_data=(X_val_scaled, Y_val_std), verbose=0)

                # Evaluate the model
                train_loss = history.history['loss'][-1]
                val_loss = history.history['val_loss'][-1]

                # Check if this is the best model
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_params = {'learning_rate': lr, 'epochs': epochs, 'batch_size': batch_size}
                    best_model = model

                # Update progress bar
                pbar.update(1)

# Print the best hyperparameters
print("Best hyperparameters:", best_params)

# Print the learned parameters
weights, biases = best_model.layers[0].get_weights()
print("Learned weights:", weights)
print("Biases:", biases)

# Print the final epoch loss on the training and validation datasets
print("Training loss:", train_loss)
print("Validation loss:", val_loss)

# Calculate the percentage difference between the training and validation losses
percentage_difference = abs(train_loss - val_loss) / train_loss * 100
print("Percentage difference between training and validation loss:", percentage_difference)

# Check if the optimal model configuration is achieved
if val_loss < 0.35:
    print("Optimal model configuration achieved.")
else:
    print("Optimal model configuration not achieved.")

---
### Step 6: Evaluation and Generalization


Now that you've determined the optimal set of hyperparameters, it's time to evaluate your optimized model on the test data to gauge its performance in real-world scenarios, commonly known as inference.

### <span style="color:chocolate">Exercise 12:</span> Computing MSE (10 points)

1. Calculate MSE on both (X_train_std, Y_train_std) and (X_test_std, Y_test_std) datasets. Hint: You can utilize the <span style="color:chocolate">model.evaluate()</span> method provided by tf.keras.

2. Does the model demonstrate strong generalization capabilities? Provide an explanation based on your observations.

4. Generate a plot to visualize the accuracy of the predictions. Plot the actual (observed) Y_test values on the x-axis and the predicted Y_test values on the y-axis. Additionally, include a 45-degree line in the plot for reference. Ensure that the plot contains appropriate axis labels and a title. Provide commentary on the model's fit based on this visualization. Hint: You can utilize the <span style="color:chocolate">model.predict()</span> method available in tf.keras.

In [ ]:
# YOUR CODE HERE
epochs = 100
batch_size = 16
learning_rate = 0.02

best_model = build_model(num_features = num_o_features, learning_rate = learning_rate)
best_model.fit(X_train_std, Y_train_std, batch_size=batch_size, epochs=epochs, validation_data=(X_val_std, Y_val_std))

train_mse = best_model.evaluate(X_train_std, Y_train_std)
print(f"Training MSE: {train_mse}")

test_mse = best_model.evaluate(X_test_std, Y_test_std)
print(f"Test MSE: {test_mse}")

#this model does demonstrate strong generalization because the mse between training  and test  are close Training MSE: 0.2644072473049164  Test MSE: 0.25394168496131897

In [ ]:
# Ensure you predict using the test set
Y_pred = best_model.predict(X_test_std)

# Flatten the Y_pred array if necessary
if Y_pred.shape != Y_test_std.shape:
    Y_pred = Y_pred.flatten()

# Plot the observed vs predicted values
plt.figure(figsize=(8, 6))
plt.scatter(Y_test_std, Y_pred, color='blue', alpha=0.5, label='Predicted vs Actual')
plt.plot([min(Y_test_std), max(Y_test_std)], [min(Y_test_std), max(Y_test_std)], color='red', linestyle='--', label='45-degree line')

# Add labels and title
plt.xlabel('Observed Y_test values')
plt.ylabel('Predicted Y_test values')
plt.title('Accuracy of the Predictions')
plt.legend()
plt.show()

Provide commentary on the model's fit based on this visualization. Hint: You can utilize the model.predict() method available in tf.keras.

there is alot of clustering near the origin and as we get further away the predictions start to deviate meaning that there might be a certain price range where our model is really good at predicting. Overall our predictions are near our 45 degree line meaning it  good model performance with some deviations.

----
### <span style="color:chocolate">Bonus question</span> (20 points)

In Exercise 12, you reported an aggregated MSE. Let's revisit the exercise by:

1. Conducting a subgroup model evaluation. More precisely, compute the test data MSE based on various car subgroups such as make, engine size, fuel type, etc.

2. Answering the question: is the model "fair" to your chosen car subgroups in our data?